# Data Exploration

In this notebook describe your data exploration steps.

## Install dependencies

In [2]:
%pip install pandas
%pip install 'SQLAlchemy==1.4.46'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'SQLAlchemy==1.4.46'"


# Import Modules

importing all the needed python libraries.


In [2]:
import subprocess
import zipfile
import os
import shutil
import pandas as pd
import sqlite3
import requests
from io import StringIO
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot

## Download datasets

In [6]:
datasets = [
    {"dataset": "berkeleyearth/climate-change-earth-surface-temperature-data", "file": "GlobalLandTemperaturesByCountry.csv"},
    {"dataset": "unitednations/global-food-agriculture-statistics", "file": "fao_data_crops_data.csv"}
]


# Function to download datasets from Kaggle
def download_datasets():
    for ds in datasets:
        os.system(f"kaggle datasets download -d {ds['dataset']} -p /tmp")
        os.system(f"unzip -o /tmp/{ds['dataset'].split('/')[-1]}.zip -d /tmp")
download_datasets()



### Look at the first table data

In [10]:

file_path = f"/tmp/GlobalLandTemperaturesByCountry.csv"
df = pd.read_csv(file_path)
print(df.head)

  

<bound method NDFrame.head of                 dt  AverageTemperature  AverageTemperatureUncertainty  \
0       1743-11-01               4.384                          2.294   
1       1743-12-01                 NaN                            NaN   
2       1744-01-01                 NaN                            NaN   
3       1744-02-01                 NaN                            NaN   
4       1744-03-01                 NaN                            NaN   
...            ...                 ...                            ...   
577457  2013-05-01              19.059                          1.022   
577458  2013-06-01              17.613                          0.473   
577459  2013-07-01              17.000                          0.453   
577460  2013-08-01              19.759                          0.717   
577461  2013-09-01                 NaN                            NaN   

         Country  
0          Åland  
1          Åland  
2          Åland  
3          Åland 

  ### Transformation of first table

1. first remove NaN and columns with incomplete value 
2. change the format of dt column to datatime for better working with it
3. taking the year value for comparison with other table since it is based on year
4. filter the years more than 1961 since other table is only for the data above the given data
5. drop the uncertainity column
6. grouping the year and country to calculate the average tempreture of every country at the available years
7. change the name of average temperature to average_yearly_temperature for better clarification

In [11]:

df = df.dropna()
    # Fix errors or apply any transformations as needed
 # Load the data
  
df = pd.read_csv(file_path)

    # Convert the 'dt' column to datetime format
df['dt'] = pd.to_datetime(df['dt'])

    # Extract the year from the 'dt' column
df['year'] = df['dt'].dt.year

    # Filter the data to include only years greater than 1961
df_filtered = df[df['year'] > 1961]

    # Drop the 'AverageTemperatureUncertainty' column
df_filtered = df_filtered.drop(columns=['AverageTemperatureUncertainty'])

    # Group by year and country, then calculate the average temperature
df_grouped = df_filtered.groupby(['year', 'Country'])['AverageTemperature'].mean().reset_index()

    # Rename the columns for clarity
df_grouped.rename(columns={'AverageTemperature': 'average_yearly_temperature'}, inplace=True)
print(df_grouped.head)
df_first = df_grouped

<bound method NDFrame.head of        year         Country  average_yearly_temperature
0      1962     Afghanistan                   13.768667
1      1962          Africa                   24.194500
2      1962         Albania                   13.068750
3      1962         Algeria                   23.182417
4      1962  American Samoa                   26.627667
...     ...             ...                         ...
12631  2013  Western Sahara                   23.744250
12632  2013           Yemen                   28.129750
12633  2013          Zambia                   21.196000
12634  2013        Zimbabwe                   20.710750
12635  2013           Åland                    6.229750

[12636 rows x 3 columns]>


### Look at the second table data


In [12]:

file_path = f"/tmp/fao_data_crops_data.csv"
df = pd.read_csv(file_path)
print(df.head)

<bound method NDFrame.head of         country_or_area                                       element_code  \
0            Americas +                                                 31   
1            Americas +                                                 31   
2            Americas +                                                 31   
3            Americas +                                                 31   
4            Americas +                                                 31   
...                 ...                                                ...   
2255344              Fc                                    Calculated Data   
2255345              A   May include official, semi-official or estimat...   
2255346              NR                            Not reported by country   
2255347              F                                        FAO Estimate   
2255348              *                                   Unofficial figure   

                element    year u

### Info about the transformation

1. Droping the NaN values
2. checking the unique elements so that we can find the good value to messure the difference
3. choosing the most suitable elemnt which is the yield through hg/ha unit. 
4.Droping the useless elements

In [26]:
df = pd.read_csv(file_path)
    # Example transformation: drop missing values
df = df.dropna()
    # Fix errors or apply any transformations as needed

    #Listing the units and ways of estimating crops
print(df['element'].unique())
print(df['unit'].unique())

filtered_df_check = df[df['element'] == 'Yield']
print(filtered_df_check['unit'].unique())
    # Define the values you need in 'column2'
values_needed = ['Hg/Ha']

    # Filter the DataFrame
filtered_df = df[df['unit'].isin(values_needed)]
print(filtered_df['element'].unique())
    # Columns to drop
columns_to_drop = ['element_code', 'value_footnotes']

    # Drop the specified columns
df_dropped = filtered_df.drop(columns=columns_to_drop)
df_second = df_dropped
print(df_second.head)
    # print(df_dropped)

['Area Harvested' 'Yield' 'Production Quantity' 'Seed'
 'Gross Production 1999-2001 (1000 I$)'
 'Net Production 1999-2001 (1000 I$)' 'Gross PIN (base 1999-2001)'
 'Grs per capita PIN (base 1999-2001)' 'Net PIN (base 1999-2001)'
 'Net per capita PIN (base 1999-2001)']
['Ha' 'Hg/Ha' 'tonnes' '1000 Int. $' 'Int. $']
['Hg/Ha']
['Yield']
<bound method NDFrame.head of         country_or_area element    year   unit    value          category
47           Americas +   Yield  2007.0  Hg/Ha  10921.0  agave_fibres_nes
48           Americas +   Yield  2006.0  Hg/Ha  10921.0  agave_fibres_nes
49           Americas +   Yield  2005.0  Hg/Ha  10921.0  agave_fibres_nes
50           Americas +   Yield  2004.0  Hg/Ha  11188.0  agave_fibres_nes
51           Americas +   Yield  2003.0  Hg/Ha  10842.0  agave_fibres_nes
...                 ...     ...     ...    ...      ...               ...
2255291         World +   Yield  1965.0  Hg/Ha  51952.0    yautia_cocoyam
2255292         World +   Yield  1964.0  Hg

### Merging tables
Merging two tables based on date and country with inner join

In [ ]:
# Merge DataFrames
result = pd.merge(df_first, df_second, left_on=['country_or_area', 'year'], right_on=['Country', 'year'])

print(result)


### Data exploration
Print some basic information about the data. Your data exploration would continue here.

In [6]:
df.info()

df['Status'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6519 entries, 0 to 6518
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   EVA_NR          6519 non-null   int64  
 1   DS100           6519 non-null   object 
 2   IFOPT           6512 non-null   object 
 3   NAME            6519 non-null   object 
 4   Verkehr         6519 non-null   object 
 5   Laenge          6519 non-null   float64
 6   Breite          6519 non-null   float64
 7   Betreiber_Name  6517 non-null   object 
 8   Betreiber_Nr    5395 non-null   float64
 9   Status          24 non-null     object 
dtypes: float64(3), int64(1), object(6)
memory usage: 509.4+ KB


array(['neu', None], dtype=object)